# PHS564 — Lecture 11 (Student)
## Time-varying treatment and confounding: marginal structural models (MSMs) (MIMIC-IV Demo)

### Learning goals
- Recognize time-varying confounding affected by prior treatment.
- Construct stabilized weights over time:
- treatment weights \(W_A\)
- censoring weights \(W_C\)
- combined weights \(W=W_A	imes W_C\)
- Fit a simple MSM (weighted pooled logistic/GLM) and interpret the parameter as a **marginal causal effect** under assumptions.

### Required reading
- Hernán & Robins, MSM sections (typically Chapter 12 continuation / longitudinal chapters).


### Setup

This notebook is designed to run **locally** or in **Google Colab**.

**Colab workflow (recommended):**
1) Clone the course repo (ask the instructor for the GitHub URL).
2) Install requirements.
3) Run the notebook top-to-bottom.

> If you opened this notebook directly from GitHub in Colab (without cloning),
> relative paths will not work. Clone first.


In [ ]:
from __future__ import annotations

import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Reproducibility
RNG = np.random.default_rng(564)

# Locate repo root (works when running from lectures/Lxx.../student or /instructor)
THIS_DIR = Path.cwd()
REPO_ROOT = THIS_DIR
for _ in range(4):
    if (REPO_ROOT / "requirements.txt").exists() or (REPO_ROOT / "README.md").exists():
        break
    REPO_ROOT = REPO_ROOT.parent

DATA_DIR = REPO_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROC_DIR = DATA_DIR / "processed"

print("Working directory:", THIS_DIR)
print("Repo root:", REPO_ROOT)
print("Processed data dir exists:", PROC_DIR.exists())


## Data
Expected file: `data/processed/cohort_L11_msm_longitudinal.parquet` (or `.csv`).

Assumed **long** format with columns like:
- `id`, `t`
- treatment `A_t`
- time-varying confounders `L_t` (may be multiple)
- outcome `Y` at end (or `Y_t`)

This notebook is **diagnostics-first**: you will compute stabilized weights using a provided scaffold, then **diagnose** and **interpret** the MSM estimate.


In [ ]:
# Statsmodels for regression (logit/ols); installed via requirements.txt
import statsmodels.api as sm
import statsmodels.formula.api as smf
parquet_path = PROC_DIR / "cohort_L11_msm_longitudinal.parquet"
csv_path = PROC_DIR / "cohort_L11_msm_longitudinal.csv"

if parquet_path.exists():
    df = pd.read_parquet(parquet_path)
elif csv_path.exists():
    df = pd.read_csv(csv_path)
else:
    raise FileNotFoundError("Missing cohort file for L11. Run `python data/download_data.py`.")

df.head()

### Variables


In [ ]:
ID = "id"
T = "t"
A = "A_t"  # TODO: rename if needed
Y = "Y"    # end-of-follow-up outcome column
L_t = ["L_t"]  # TODO: list time-varying confounders
L0 = ["age","sex"]  # TODO: baseline covariates

cols_needed = [ID,T,A,Y] + L0 + L_t
missing = [c for c in cols_needed if c not in df.columns]
missing

## Part A — Stabilized treatment weights (scaffold)
You should NOT re-derive the formulas here. Focus on model specification and diagnostics.


In [ ]:
def stabilized_treatment_weights(data: pd.DataFrame) -> pd.DataFrame:
    d = data.copy()
    d = d.sort_values([ID, T]).reset_index(drop=True)

    # Denominator model: Pr(A_t=1 | past A, past L, baseline L0)
    # Numerator model:   Pr(A_t=1 | past A, baseline L0)
    # NOTE: The exact covariate history depends on your extract. This is a scaffold.

    # Create simple lag features (example: lagged treatment and lagged confounder)
    d["A_lag1"] = d.groupby(ID)[A].shift(1).fillna(0)
    for l in L_t:
        d[f"{l}_lag1"] = d.groupby(ID)[l].shift(1)

    denom_covs = ["A_lag1"] + L0 + [f"{l}_lag1" for l in L_t]
    num_covs   = ["A_lag1"] + L0

    denom_formula = A + " ~ " + " + ".join([c for c in denom_covs if c in d.columns])
    num_formula   = A + " ~ " + " + ".join([c for c in num_covs if c in d.columns])

    denom = smf.logit(denom_formula, data=d).fit(disp=False)
    num = smf.logit(num_formula, data=d).fit(disp=False)

    p_denom = denom.predict(d)
    p_num = num.predict(d)

    # Avoid division by zero
    eps = 1e-6
    p_denom = np.clip(p_denom, eps, 1-eps)
    p_num = np.clip(p_num, eps, 1-eps)

    w_t = np.where(d[A]==1, p_num/p_denom, (1-p_num)/(1-p_denom))
    d["sw_treat_t"] = w_t
    d["sw_treat"] = d.groupby(ID)["sw_treat_t"].cumprod()
    return d

d = stabilized_treatment_weights(df)
d[[ID,T,A,"sw_treat_t","sw_treat"]].head()

### TODO A1 — Diagnose weights
Plot `sw_treat` and choose a truncation rule.


In [ ]:
plt.figure()
plt.hist(d["sw_treat"], bins=80)
plt.xlabel("stabilized treatment weight (cumulative)")
plt.ylabel("count")
plt.title("MSM weights before truncation")
plt.show()

lo, hi = d["sw_treat"].quantile([0.01,0.99]).to_list()
d["sw_treat_trunc"] = d["sw_treat"].clip(lo,hi)
(lo,hi)

## Part B — Fit an MSM (simple)
Example MSM: E[Y] = β0 + β1 * cumulative_treatment.

Your extract may define treatment history differently; adapt as needed.


In [ ]:
# Example: cumulative treatment exposure up to time t
d["A_cum"] = d.groupby(ID)[A].cumsum()

# Use the last time point per subject for end-of-follow-up Y
last = d.sort_values([ID,T]).groupby(ID).tail(1).copy()

# TODO: define the MSM formula
msm_formula = Y + " ~ A_cum"  # or Y ~ A_t (if outcome is time-specific)

msm = smf.ols(msm_formula, data=last).fit(weights=last["sw_treat_trunc"])
msm.summary().tables[0]

### TODO B1 — Compare to naïve (unweighted) estimate


In [ ]:
naive = smf.ols(msm_formula, data=last).fit()
{"beta1_weighted": float(msm.params.get("A_cum", np.nan)),
 "beta1_naive": float(naive.params.get("A_cum", np.nan))}

## Reflection
1) What does the MSM coefficient represent here (marginal vs conditional)?
2) Which part is harder: computing weights or interpreting the estimate?
